# Tokenizer

In [3]:
import pandas as pd
import numpy as np

# load data
TRAIN_PATH = 'process_data/train.csv'
DEV_PATH = 'process_data/dev.csv'

df_train = pd.read_csv(TRAIN_PATH)
df_dev = pd.read_csv(DEV_PATH)

In [6]:
#Parameters of tokenizer
MAX_SEQUENCE_LENGTH = 128

In [ ]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense

# Tokenization
tokenizer = Tokenizer()
tokenizer.fit_on_texts(df_train['text'])
X = tokenizer.texts_to_sequences(df_train['text'])
X = pad_sequences(X, maxlen=MAX_SEQUENCE_LENGTH)

# Model

In [1]:
import tensorflow as tf

In [12]:
import tensorflow as tf
import tensorflow_addons as tfa
from tensorflow.keras.layers import Input, Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.models import Model

def build_bilstm_crf_model(input_dim, output_dim, input_length, num_classes):
    
    input_layer = Input(shape=(input_length,))
    embedding_layer = Embedding(input_dim=input_dim, output_dim=output_dim, input_length=input_length)(input_layer)
    bilstm_layer = Bidirectional(LSTM(units=output_dim, return_sequences=True))(embedding_layer)
    crf_layer = tfa.layers.CRF(num_classes, name='sparse_categorical_crossentropy')(bilstm_layer)

    model = Model(inputs=input_layer, outputs=crf_layer)
    # model.compile(optimizer='adam', loss=tfa.losses.crf_loss, metrics=[tfa.metrics.crf_accuracy])

    return model


In [13]:
input_dim = 128
output_dim = 64
input_length = 128
num_classes = 11

model = build_bilstm_crf_model(input_dim, output_dim, input_length, num_classes)
model.summary()

Model: "model_6"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_7 (InputLayer)        [(None, 128)]             0         
                                                                 
 embedding_6 (Embedding)     (None, 128, 64)           8192      
                                                                 
 bidirectional_6 (Bidirecti  (None, 128, 128)          66048     
 onal)                                                           
                                                                 
 sparse_categorical_crossen  [(None, 128),             1562      
 tropy (CRF)                  (None, 128, 11),                   
                              (None,),                           
                              (11, 11)]                          
                                                                 
Total params: 75802 (296.10 KB)
Trainable params: 75802 (29